# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
import numpy as np
import torch
import torchvision
import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
import sys
import torch.optim as optim
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from tqdm.notebook import tqdm
from torchvision.transforms import Resize, Normalize, ToTensor, Compose
import random 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

In [2]:
!wget https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip
!unzip -q dataset.zip

--2021-11-21 10:00:10--  https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/33l8lp62rmvtx40/dataset.zip [following]
--2021-11-21 10:00:10--  https://www.dropbox.com/s/raw/33l8lp62rmvtx40/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf32c3ab9173c0913cb9338f2ca.dl.dropboxusercontent.com/cd/0/inline/BaYYnwDI5OjUEmjD67zcYFHwUAeYXiSWI-vzaY7rVzWw8kdE-SEAXQEcYwNQvjNcMUs2mDsbVPPOM_tVrSmvPmTMaYw-L_mxg_uXDaKYU_IhFmW3yz-DPzWUD4H5HAtVEPIQWByDioJaK25zzV5aoZ1w/file# [following]
--2021-11-21 10:00:10--  https://ucf32c3ab9173c0913cb9338f2ca.dl.dropboxusercontent.com/cd/0/inline/BaYYnwDI5OjUEmjD67zcYFHwUAeYXiSWI-vzaY7rVzWw8kdE-SEAXQEcYwNQvjNcMUs2mDsbVPPOM_tVrSmvPmTMaYw-L

### Подготовка данных

In [3]:
train_transform = Compose([
        ToTensor(),
        torchvision.transforms.RandomHorizontalFlip(), 
        torchvision.transforms.ColorJitter(),
    ])
val_transform = Compose([
        ToTensor()
    ])

train_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/train", transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder("./dataset/dataset/val", transform=val_transform)

In [4]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=8)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                          shuffle=False, num_workers=8)

In [ ]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [5]:
def train_epoch(model, data_loader, optimizer, criterion, scheduler, return_losses=False, device="cuda:0"):
    model = model.to(device).train()  # переводим модель на ГПУ + указываем, что сейчас обучение
    total_loss = 0  # лосс за эпоху
    num_batches = 0  # число батчей, по которым мы посчитали уже
    all_losses = []  # лоссы по батчам
    total_predictions = np.array([])  # наши ответы по батчам
    total_labels = np.array([])  # правильные ответы по батчам

    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            # переводим на ГПУ
            images = images.to(device)
            labels = labels.to(device)

            predicted = model(images)  # <-> model.forward(images), т.е. предсказания
            loss = criterion(predicted, labels)  # считаем лосс
            
            # обновляем веса
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # обновляем значения для прогресс-бара
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)

            # записываем результаты
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())

    # записывам метрики за эпоху
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    print("Accuracy:", (total_predictions == total_labels).mean())

    if return_losses:
        return metrics, all_losses
    else:
        return metrics
        

def predict(model, data_loader, criterion, device="cuda:0"):
    model = model.eval()  # указываем, что сейчас предсказания
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])

    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()

            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)

            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1

    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    print("Accuracy:", (total_predictions == total_labels).mean())
    return metrics, total_predictions, total_labels


def train(model, epochs, train_data_loader, validation_data_loader, optimizer, criterion, scheduler, device="cuda:0"):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # обучаем
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_epoch(
            model=model,
            data_loader=train_data_loader,
            optimizer=optimizer,
            return_losses=True,
            criterion=criterion,
            scheduler=scheduler,
            device=device
        )
        # сохраняем результаты
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics["loss"])
        # предсказываем
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics, total_predictions, total_labels = predict(
                model=model,
                data_loader=validation_data_loader,
                criterion=criterion
            )
        # сохраняем результаты
        epoch_eval_losses.append(validation_metrics["loss"])
        scheduler.step(validation_metrics['loss'])

### Обучение модели, запуски экспериментов

In [ ]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(512, 200)  # меняем линейный слой, нам надо 200 выходов
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), padding=(1, 1)) # попробуем переделать первый слой
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()  # т.к. не делали softmax
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')  # выбрала первый попавшийся

Простой тест на проверку правильности написанного кода


In [ ]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

  0%|          | 0/313 [00:00<?, ?it/s]

tests passed


In [23]:
train(model, 6, train_dataloader, val_dataloader, optimizer, criterion, scheduler, device=device)

Train Epoch: 0


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.45298
Validation Epoch: 0


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.3715
Train Epoch: 1


...

Accuracy: 0.4422
Train Epoch: 5


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.65729
Validation Epoch: 5


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.4519


### Проверка полученной accuracy

In [26]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print(accuracy)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * accuracy / 0.44)))

  0%|          | 0/313 [00:00<?, ?it/s]

0.4519
Оценка за это задание составит 10 баллов


### Отчёт об экспериментах 

Большинство экспериментов я провела, когда у меня была неправильно написанная функция train (я была уверена, что у меня получаются странные результаты из-за этого). Поэтому они получились бесполезными. Потом я наконец переписала правильно, поняла, что писать с нуля сетку, как в первом дз, эффекта нет, и взяла за основу семинар 5, где использовали предобученные модели. Только предобучать не стала, взяла только саму модель --- и обучила сама. Как и показывали на семинаре, заменила линейный слой, т.к. мне нужно другое количество выходов. Сразу взяла scheduler --- взяла первый попавшийся, ни разу не меняла на другой. Все остальное взяла стандартное, как всегда брала (оптимизатор --- Adam с lr = 1e - 4, метрику --- кросс-энтропию (тут потому что я не делала softmax, а мне по принципу работы нужно. Знаю, что это тоже оптимизирует работу (не считать два раза softmax)). Аугментации сразу использовала. Еще какую-то сначала добавляла, но потом убрала (во время неправильной функции train, так что как она влияла на результат --- не знаю. У меня все получилось без этого преобразования, и больше исследовать ничего не захотелось). Больше я ничего не делала, только сто раз запустила ячейку с обучением, чтобы убедиться, что у меня и правда такие показатели. Привела код для нескольких экспериментов --- самый простой вариант, с резнет50, а также с изменением первого слоя (пробовала с резнет50 и резнет18). Некоторые результаты во втором ноутбуке (запускала с нескольких аккаунтов, чтобы быстрее было!)

Было больше экспериментов со второй частью. Пробовала добавлять слои, недообучать некоторые последние (качество только падало, видимо, много удаляла лишнего или не то добавляла), брать модели с большим количеством слоев (с слишком большим, видимо, было слишком для моего датасета) и т.д. Забила. Потом у меня получилась первая часть. Я стала к ней писать отчет и решила еще описать, что попробовала для второй части. На воодушевлении решила попробовать самое простое, что пришло в голову, и оно сработало)) хотя раньше не получалось пробить выше 0.67. Код с результатом для второй части в отдельном ноутбуке, потому что для скорости я запускала его с другого аккаунта)) К сожалению, коды с разными вариациями не сохранились, потому что я думала, что ничего не получилось, и все удалила. Но там было все то, что я описывала сверху, без особых изысков --- ничего удивительного!

Кажется, все рассказала! Я сделала семь новых аккаунтов в гугле, потому что потратила все часы ГПУ...

In [ ]:
# 1 версия
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(512, 200)  # меняем линейный слой, нам надо 200 выходов
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()  # т.к. не делали softmax
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')  # выбрала первый попавшийся

In [ ]:
# 2 версия
model = torchvision.models.resnet50(pretrained=False)
model.fc = nn.Linear(2048, 200)  # меняем линейный слой, нам надо 200 выходов
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()  # т.к. не делали softmax
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')  # выбрала первый попавшийся

In [15]:
# 3 версия (итоговая)
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(512, 200)  # меняем линейный слой, нам надо 200 выходов
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), padding=(1, 1)) # попробуем переделать первый слой
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()  # т.к. не делали softmax
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')  # выбрала первый попавшийся

In [9]:
# 3 версия, но с резнет50
train(model, 5, train_dataloader, val_dataloader, optimizer, criterion, scheduler, device=device)

Train Epoch: 0


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.11994
Validation Epoch: 0


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.1811
Train Epoch: 1


...

Accuracy: 0.3443
Train Epoch: 4


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.40949
Validation Epoch: 4


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.3672


In [ ]:
# 2 версия
train(model, 3, train_dataloader, val_dataloader, optimizer, criterion, scheduler, device=device)

Train Epoch: 0


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.41975
Validation Epoch: 0


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.3045
Train Epoch: 1


...

Accuracy: 0.31
Train Epoch: 2


  0%|          | 0/1563 [00:00<?, ?it/s]

Accuracy: 0.51788
Validation Epoch: 2


  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.3133
